# Imports

In [1]:
import numpy as np
from scipy.optimize import minimize
import random
import multiprocessing

import cvxpy as cp

In [2]:
np.random.seed(1)

# Create random Reachable sets

In [3]:
c = 5 # cache capacity
k = 5 # each user should be in at most K reachable sets
m = 2 # at most M items can be initially recommended to each user
L = 5 # the total number of items that the user will receive either through 
# direct recommendation or indirectly (because it is in reachable sets of users to whom items 
#have been recommended) should not be more than L
number_of_items = 10
number_of_users = 12
items= np.array(range(0,number_of_items)) # videos
users = np.array(range(0,number_of_users)) # users
# initialize x0 for the optimization
x = np.full(len(items),0.5)
y = np.full((len(items), len(users)),0.5)
x0 = np.concatenate([x.flatten(), y.flatten()])
# Random Reachable sets
reachable_sets = np.array(np.array([[np.array(random.sample(list(users), random.choice(users))) for user in users] for i in items]))
I = np.zeros((len(items), len(users), len(users)))
for i in items:
    for u in users:
        for v in reachable_sets[i][u]:
            I[i][u][v] = 1

# Objective function

In [4]:
def objective(z):
    x = z[:len(items)]
    y = z[len(items):].reshape(len(items),len(users))
    
    #The first term in (1) counts the users to which an item is 
    #initially recommended and the users in the ensuing reachable sets
    #first_term = sum([(1+len(reachable_sets[i][u])) * y[i][u] for u in users])
    #The second term in (1) is due to the overlap of reachable sets Riu,Riv
    #second_term = 0.5*sum([sum([len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))*y[i][u]*y[i][v] for v in users if v!=u]) for u in users])
    # Ekfonisi
    #return -sum([x[i]*(sum([(1+len(reachable_sets[i][u])) * y[i][u] for u in users]) - 0.5*sum([sum([len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))*y[i][u]*y[i][v] for v in users if v!=u]) for u in users])) for i in items])
    # Paper
    return -sum([x[i]*sum([(1+len(reachable_sets[i][u])) * y[i][u] - 0.5*sum([len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))*y[i][u]*y[i][v] for v in users if v!=u]) for u in users]) for i in items])

# Constraint (2) is the cache capacity constraint

In [5]:
def constraint2(z):  
    x = z[:len(items)]
    
    return sum([x[i] for i in items]) - c

# Constraint (3) says that at most M items can be initially recommended to each user

In [6]:
def constraint3(z):  
    y = z[len(items):].reshape(len(items),len(users))
    
    return [(-sum([y[i][u] for i in items]))+m for u in users]

# Constraint (4) says that each user should be in at most K reachable sets these capture content consumption, attention span or time constraints

In [7]:
def constraint4(z):  
    y = z[len(items):].reshape(len(items),len(users))
    
    return [-sum([sum([I[i][u][v]*y[i][u] for u in users]) for i in items])+k for u in users]

# Constraint (6) says that the total number of items that the user will receive either through direct recommendation or indirectly (because it is in reachable sets of users to whom items have been recommended) should not be more than L

In [8]:
def constraint6(z):
    y = z[len(items):].reshape(len(items),len(users))
    

    return [-sum([y[i][u] for i in items]) - sum([sum([I[i][u][v]*y[i][u] for v in users]) for i in items])+L for u in users]

# Solve

In [9]:
con2 = {'type': 'eq', 'fun': constraint2} 
con3 = {'type': 'ineq', 'fun': constraint3} 
con4 = {'type': 'ineq', 'fun': constraint4} 
con6 = {'type': 'ineq', 'fun': constraint6}
cons = ([con2, con6])
# Bounds
b = (0.0,1.0)
bnds = [b for i in range(len(x0))]

# show initial objective
print('Initial Objective: ' + str(-1*objective(x0)))

solution = minimize(objective, x0, method='SLSQP', bounds=bnds, constraints=cons)

Initial Objective: -28.5


In [10]:
x = solution.x

# show final objective
print('Final Objective: ' + str(-1*objective(x)))

Final Objective: 55.75198412743564


In [11]:
solution

     fun: -55.75198412743564
     jac: array([ -9.99987316,   0.        , -14.12512875,   0.        ,
         0.        ,   0.        , -11.38888741,  -8.33333349,
       -11.90476179,   0.        ,  -3.        ,  -6.3125639 ,
        -1.0625639 ,  -3.        ,  -2.        ,  -6.0625639 ,
        -3.25      ,  -3.5       ,  -2.5       ,  -1.        ,
        -1.        ,  -3.3125639 ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,  -2.12495708,  -2.12495708,
        -2.74991417,  -4.        ,  -3.        ,  -3.24991417,
        -2.5       ,  -2.        ,  -1.        ,  -3.        ,
        -0.62495708,  -2.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.     

In [12]:
solution.x.shape

(130,)

In [13]:
x = solution.x[:len(items)].round()
y = solution.x[len(items):].reshape(len(items),len(users)).round()

# Solution

## Items in cache

In [14]:
x

array([1., 0., 1., 0., 0., 0., 1., 1., 1., 0.])

In [15]:
x.sum()

5.0

In [16]:
x.shape

(10,)

## Recommended items

In [17]:
y

array([[1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [18]:
y.sum(axis=0) # items recommended to each user

array([1., 2., 2., 2., 3., 1., 2., 2., 2., 2., 1., 1.])

In [19]:
y.shape

(10, 12)

In [20]:
# x = cp.Variable(x.shape)
# y = cp.Variable(y.shape)
# prob = cp.Problem(cp.Maximize(cp.sum(x*cp.sum())),
#                  [x<=1,
#                   x>=0,
#                   y<=1,
#                   y>=0
                     
# #                   sum([x[i] for i in items]) == c,
# #                  [sum([y[i][u] for i in items]) <= m for u in users],
# #                  [sum([sum([I[i][u][v]*y[i][u] for u in users]) for i in items]) <= k for u in users]
#                   ])
# prob.solve()